### Hyperparameter tuning of Deep learning models in Keras

In [1]:
# Import libraries for convolutional network model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [2]:
!pip install aisaratuners 
from aisaratuners.aisara_keras_tuner import Hp, HpOptimization

     |████████████████████████████████| 13.2MB 248kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [3]:
(img_train, y_train), (img_test, y_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
y_test

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

In [5]:
# Normalize pixel values between 0 and 1
x_train = img_train.astype('float32') / 255.0
x_test = img_test.astype('float32') / 255.0

img_rows, img_cols = 28, 28
if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [6]:
y_train=to_categorical(y_train,)
y_test=to_categorical(y_test)

In [7]:
# define Hps:
hps = Hp()
hp_1 = hps.numrange(name='num_layers_conv',min=2,max=10)
hp_2 = hps.numrange(name='nodes_dense',min=64,max=512)
hp_3 = hps.numrange(name='lr',min=0.0001,max=0.01, type='log')

In [8]:
# define keras model and return model history
def model_builder(comp, k):
    model = keras.Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape,padding='same'))
    for layer in range(comp['num_layers_conv'][k]):
        model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(comp['nodes_dense'][k], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(comp['nodes_dense'][k], activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(comp['lr'][k]),metrics=['acc'])
    history = model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))
    return model, history

In [9]:
# run optimizer
demo = HpOptimization(hps, model_builder, ['val_acc','val_loss'], ['max','min'], num_trials=3, rounds=1, mode='p')
%time demo.run_opti()

For commercial use, you can obtain our API from https://rapidapi.com/aisara-technology-aisara-technology-default/api/aisara-hyperparameter-tuning
If you are a private user, set the mode parameter in HpOptimization class to "p".

Round-1:

  Run-1:
Epoch 1/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.4502 - acc: 0.8368 - val_loss: 0.3441 - val_acc: 0.8738
Epoch 2/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.3303 - acc: 0.8788 - val_loss: 0.3364 - val_acc: 0.8785
Epoch 3/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2968 - acc: 0.8913 - val_loss: 0.3123 - val_acc: 0.8897
Epoch 4/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2647 - acc: 0.9000 - val_loss: 0.2844 - val_acc: 0.8958
Epoch 5/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2509 - acc: 0.9075 - val_loss: 0.3095 - val_acc: 0.8968

  Run-2:
Epoch 1/5
1875/1875 [==============================] - 22s 12ms/step - l

In [10]:
demo.best_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 28, 64)       